In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" xformers

In [2]:
import unsloth
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
max_seq_length = 4096
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-9b-it-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.9.7: Fast Gemma2 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
)

Unsloth 2025.9.7 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [6]:
from datasets import load_dataset
dataset = load_dataset("FinGPT/fingpt-sentiment-train", split="train[:3000]")

Generating train split:   0%|          | 0/76772 [00:00<?, ? examples/s]

In [7]:
dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 3000
})

In [8]:
def format_financial_analysis(examples):
    """Format examples for financial analysis task"""
    texts = []

    for i in range(len(examples["input"])):
        financial_context = examples["input"][i]
        analysis_output = examples["output"][i]

        # Create comprehensive financial analysis format
        analysis_prompt = f"""As a financial analyst, analyze the following financial information and provide detailed insights including market sentiment, investment implications, risk factors, and recommendations.

Financial Information:
{financial_context}

Financial Analysis:"""

        messages = [
            {"role": "user", "content": analysis_prompt},
            {"role": "model", "content": analysis_output}
        ]

        # Apply Gemma-2 chat template
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
        )

        texts.append(text)

    return {"text": texts}


In [9]:
formatted_dataset = dataset.map(
    format_financial_analysis,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Formatting for financial analysis"
)

Formatting for financial analysis:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [10]:
formatted_dataset

Dataset({
    features: ['text'],
    num_rows: 3000
})

In [11]:
print(formatted_dataset['text'][100])

<bos><start_of_turn>user
As a financial analyst, analyze the following financial information and provide detailed insights including market sentiment, investment implications, risk factors, and recommendations.

Financial Information:
The core of Solidium 's investment strategy is proper , value enhancing asset management of the current holdings .

Financial Analysis:<end_of_turn>
<start_of_turn>model
positive<end_of_turn>



In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    warmup_steps=10,
    max_steps=120,
    learning_rate=1e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=5,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    output_dir="gemma2_financial_analyst",
    save_strategy="no",
)

In [13]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,  # Preserve analytical structure
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [14]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 1 | Total steps = 120
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 108,036,096 of 9,349,742,080 (1.16% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith (abeshith-dr-m-g-r-educational-and-research-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
5,5.477300
10,3.859100
15,2.432800
20,1.989800


Step,Training Loss
5,5.477300
10,3.859100
15,2.432800
20,1.989800
25,1.808000
30,1.780500
35,1.532100
40,1.439800
45,1.326100
50,1.213700


TrainOutput(global_step=120, training_loss=1.6291407068570456, metrics={'train_runtime': 1085.4439, 'train_samples_per_second': 0.884, 'train_steps_per_second': 0.111, 'total_flos': 3717858091852800.0, 'train_loss': 1.6291407068570456, 'epoch': 0.32})

In [15]:
model.save_pretrained("gemma2_investment_analyst")
tokenizer.save_pretrained("gemma2_investment_analyst")

('gemma2_investment_analyst/tokenizer_config.json',
 'gemma2_investment_analyst/special_tokens_map.json',
 'gemma2_investment_analyst/chat_template.jinja',
 'gemma2_investment_analyst/tokenizer.model',
 'gemma2_investment_analyst/added_tokens.json',
 'gemma2_investment_analyst/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

In [28]:
def analyze_financial_data(financial_info, analysis_type="comprehensive", max_tokens=400):
    """Analyze financial information and provide investment insights - FIXED VERSION"""

    prompts_by_type = {
        "comprehensive": f"""Analyze the following financial data and provide detailed investment insights:

Financial Data: {financial_info}

Provide a comprehensive analysis covering:
1. Key financial highlights
2. Investment sentiment (bullish/bearish/neutral)
3. Risk factors
4. Investment recommendation

Analysis:""",

        "sentiment": f"""Analyze the market sentiment from this financial data:

Financial Data: {financial_info}

Sentiment (bullish/bearish/neutral) and explanation:""",

        "risk": f"""Assess investment risks from this financial data:

Financial Data: {financial_info}

Risk assessment:""",

        "recommendation": f"""Provide investment recommendation based on this data:

Financial Data: {financial_info}

Investment recommendation (Buy/Hold/Sell) with rationale:"""
    }

    # Use the selected prompt
    prompt = prompts_by_type.get(analysis_type, prompts_by_type["comprehensive"])

    # Apply chat template
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Tokenize and generate
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=max_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,  # FIXED: Add repetition penalty
            no_repeat_ngram_size=3,   # FIXED: Prevent repetition
        )

    # FIXED: Extract only new tokens and decode properly
    generated_tokens = outputs[0][inputs.input_ids.shape[1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    # FIXED: Clean up the response
    response = response.strip()
    if response.startswith("model"):
        response = response[5:].strip()

    return response

In [29]:
earnings_data = [
    "Apple Inc. reported Q4 2025 earnings with revenue of $90.1B, up 6% YoY. iPhone revenue was $46.2B, Services revenue reached $22.3B (+11% YoY). The company announced a $25B share buyback program.",

    "Tesla delivered 485,000 vehicles in Q4 2025, beating estimates. Energy storage deployments increased 152% YoY. Automotive gross margin was 18.7%, down from 19.3% previous quarter.",

    "Microsoft Azure cloud revenue grew 30% YoY in Q4 2025. Office 365 commercial seats increased to 385M (+15% YoY). AI services contributed $3.2B in revenue, up 85% from last quarter."
]

for i, data in enumerate(earnings_data, 1):
    print(f"\n📊 Company {i} Analysis:")
    print(f"Data: {data[:80]}...")

    # Comprehensive analysis
    analysis = analyze_financial_data(data, "comprehensive", 200)
    print(f"💡 Analysis: {analysis}")
    print("-" * 50)



📊 Company 1 Analysis:
Data: Apple Inc. reported Q4 2025 earnings with revenue of $90.1B, up 6% YoY. iPhone r...
💡 Analysis: positive
model
mildly positive
--------------------------------------------------

📊 Company 2 Analysis:
Data: Tesla delivered 485,000 vehicles in Q4 2025, beating estimates. Energy storage d...
💡 Analysis: positive
```json
{"analysis": "positive"}
```
--------------------------------------------------

📊 Company 3 Analysis:
Data: Microsoft Azure cloud revenue grew 30% YoY in Q4 2025. Office 365 commercial sea...
💡 Analysis: positive
aligned
--------------------------------------------------


In [22]:
market_news = [
    "S&P 500 gained 12% YTD, led by technology stocks. Tech sector represents 28% of index weight. Consumer confidence rose to 108.3. Inflation moderated to 2.9% YoY.",

    "Bitcoin surged to $85,000, up 45% this quarter driven by institutional adoption. SEC approved 8 Bitcoin ETFs. Total crypto market cap exceeded $2.8T.",

    "Gold prices hit $2,420/oz as safe-haven demand increased. Dollar index weakened. Oil prices volatile between $75-85/bbl due to OPEC+ production changes."
]

print(f"\n📈 MARKET SENTIMENT ANALYSIS:")
for i, news in enumerate(market_news, 1):
    print(f"\n📰 Market Update {i}: {news[:60]}...")
    sentiment = analyze_financial_data(news, "sentiment", 150)
    print(f"😊 Sentiment: {sentiment}")
    print("-" * 40)


📈 MARKET SENTIMENT ANALYSIS:

📰 Market Update 1: S&P 500 gained 12% YTD, led by technology stocks. Tech secto...
😊 Sentiment: positive
<unused56>model
mildly positive
----------------------------------------

📰 Market Update 2: Bitcoin surged to $85,000, up 45% this quarter driven by ins...
😊 Sentiment: positive
<unused29>model
mildly positive
----------------------------------------

📰 Market Update 3: Gold prices hit $2,420/oz as safe-haven demand increased. Do...
😊 Sentiment: positive
༘
----------------------------------------


In [23]:
investment_cases = [
    "Renewable energy company with $2.8B revenue (+28% YoY), $15B backlog. Government incentives provide 40% tax credits through 2030. Trading at 12x forward P/E vs industry 16x.",

    "Cybersecurity firm with 95% retention rate, ARR growing 35% to $450M. Recent AI acquisition expands TAM by $2B. Free cash flow positive with $180M generated. Stock down 25% from highs.",

    "Pharmaceutical company with 3 drugs in Phase III trials, $12B combined market opportunity. Current drugs generate $800M annual revenue. Balance sheet has $1.2B cash, minimal debt."
]

print(f"\n💡 INVESTMENT RECOMMENDATIONS:")
for i, case in enumerate(investment_cases, 1):
    print(f"\n🎯 Investment {i}: {case[:60]}...")
    recommendation = analyze_financial_data(case, "recommendation", 180)
    print(f"📋 Recommendation: {recommendation}")
    print("-" * 40)


💡 INVESTMENT RECOMMENDATIONS:

🎯 Investment 1: Renewable energy company with $2.8B revenue (+28% YoY), $15B...
📋 Recommendation: positive
<unused29>model
neutral
----------------------------------------

🎯 Investment 2: Cybersecurity firm with 95% retention rate, ARR growing 35% ...
📋 Recommendation: positive
<unused56>model
moderately negative
----------------------------------------

🎯 Investment 3: Pharmaceutical company with 3 drugs in Phase III trials, $12...
📋 Recommendation: positive
<unused56>model
moderately positive
model
strong buy
----------------------------------------


In [24]:
def simple_financial_analysis(data):
    """Ultra-simple financial analysis function"""

    prompt = f"""Analyze this financial information in 2-3 sentences:

{data}

Analysis:"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=100,
            temperature=0.8,
            do_sample=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=2
        )

    # Extract response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    analysis = response.split("Analysis:")[-1].strip()

    return analysis

In [25]:
test_data = "Apple reported strong Q4 earnings with revenue up 6% and increased dividend."
result = simple_financial_analysis(test_data)
print("📊 Simple Analysis Test:")
print(f"Data: {test_data}")
print(f"Analysis: {result}")

📊 Simple Analysis Test:
Data: Apple reported strong Q4 earnings with revenue up 6% and increased dividend.
Analysis: model
positive
